In [3]:
import pandas as pd
from pybedtools import BedTool
from pybedtools.featurefuncs import TSS
import os
import numpy as np
import swifter

In [4]:
import sys
args = sys.argv  # a list of the arguments provided (str)
print("processing Chiapet", args)
rawBed= args[1]

processing Chiapet ['/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py', '-f', '/home/yudeep/.local/share/jupyter/runtime/kernel-47132032-bec1-4efb-9049-614082091d09.json']


In [5]:
col_Names=["0", "1", "2", "3","4"]


In [6]:
df212 = pd.read_csv('hicFiles/df3pe111.bed',sep='\t',names=col_Names)

In [7]:
df212.head()

,0,1,2,3,4
0,chr1,2417049,2457049,chr1_1720000_1760000_chr1_1800000_1840000,1.475869e-12
1,chr1,3979460,4019460,chr1_4480000_4520000_chr1_4520000_4560000,1.791426e-05
2,chr1,4059460,4099460,chr1_4200000_4240000_chr1_4600000_4640000,4.714114e-03
3,chr1,4059460,4099460,chr1_4520000_4560000_chr1_4600000_4640000,6.392099e-06
4,chr1,4099460,4139460,chr1_3720000_3760000_chr1_4640000_4680000,8.710911e-07


In [8]:
df213 = pd.read_csv('hicFiles/df4pe111.bed',sep='\t',names=col_Names)

In [9]:
df212 = df212.sort_values(by=['0','1'],ascending=True)
df213 = df213.sort_values(by=['0','1'],ascending=True)

In [10]:
len(df212)

443130

In [11]:
len(df213)

438830

In [12]:
df212.to_csv('df3pe11.bed',sep='\t',index=False,header=None)
df213.to_csv('df4pe11.bed',sep='\t',index=False,header=None)

In [13]:
os.system('../liftOver df4pe11.bed hg19ToHg38.over.chain.gz df4pe111.bed unmapped')

Reading liftover chains
Mapping coordinates


0

In [14]:
os.system('../liftOver df3pe11.bed hg19ToHg38.over.chain.gz df3pe111.bed unmapped')

Reading liftover chains
Mapping coordinates


0

In [15]:
def getClosestgeneWithinDistance(enhancer,genes,distance,num):
    snp = BedTool(enhancer)
    gene = BedTool(genes)
    gene.sort()  
    nearby = snp.closest(gene, d=True,k=num, output='tempchiaPet.bed')
    dfSNP = pd.read_csv(enhancer, sep='\t', header=None)
    dfGene = pd.read_csv(genes, sep='\t', header=None)
    df1 = pd.read_csv('tempchiaPet.bed', sep='\t', header=None)
    df1 = df1[df1[len(df1.columns)-1] == distance]
    os.remove('tempchiaPet.bed')
    return df1

In [16]:
df32= pd.read_csv('mart_export.txt.gz',sep='\t')

In [17]:
df32.head()

,Gene stable ID,Transcript stable ID,HGNC ID,HGNC symbol,Gene start (bp),Gene end (bp),Chromosome/scaffold name
0,ENSG00000227568,ENST00000458433,HGNC:39634,SNX18P26,18807279,18808018,13
1,ENSG00000230931,ENST00000446657,HGNC:39676,GXYLT1P1,18724415,18724904,13
2,ENSG00000204718,ENST00000330492,HGNC:39834,CNN2P12,18726706,18727414,13
3,ENSG00000231238,ENST00000652806,HGNC:42667,LINC00349,18538880,18551214,13
4,ENSG00000231238,ENST00000448748,HGNC:42667,LINC00349,18538880,18551214,13


In [18]:
print("part1")

part1


In [19]:
def isEnhancer(a,b,c):
    cc= aa[(aa[0] == a) & (aa[1].between(b,c))]
    return cc

In [20]:
def isGene(a,b,c):
    cc= bb[(bb[0] == a) & bb[1].between(b,c)]
    return cc

In [21]:
def checkIfgeneOrEnhancer(file1,file2):
    a_enhancer = getClosestgeneWithinDistance(file1,'df3pe11.bed',0,1)
    b_gene = getClosestgeneWithinDistance(file2,'df3pe11.bed',0,1)
    c_enhancer = getClosestgeneWithinDistance(file1,'df4pe11.bed',0,1)
    d_gene = getClosestgeneWithinDistance(file2,'df4pe11.bed',0,1)
    return [a_enhancer,b_gene,c_enhancer,d_gene]

In [22]:
def startPosition(df):
    return 'chr'+str(df['Chromosome/scaffold name'])

In [23]:
def getTSS(df):
    return int(df['Gene start (bp)'])-1

In [24]:
df32['0'] = df32.apply(startPosition,axis=1)
df32['1'] = df32.apply(getTSS,axis=1)
df32['2'] = df32['Gene start (bp)']
df32['3'] = df32['Gene end (bp)']
df32['4'] = df32['Gene stable ID']

In [25]:
df32.head()

,Gene stable ID,Transcript stable ID,HGNC ID,HGNC symbol,Gene start (bp),Gene end (bp),Chromosome/scaffold name,0,1,2,3,4
0,ENSG00000227568,ENST00000458433,HGNC:39634,SNX18P26,18807279,18808018,13,chr13,18807278,18807279,18808018,ENSG00000227568
1,ENSG00000230931,ENST00000446657,HGNC:39676,GXYLT1P1,18724415,18724904,13,chr13,18724414,18724415,18724904,ENSG00000230931
2,ENSG00000204718,ENST00000330492,HGNC:39834,CNN2P12,18726706,18727414,13,chr13,18726705,18726706,18727414,ENSG00000204718
3,ENSG00000231238,ENST00000652806,HGNC:42667,LINC00349,18538880,18551214,13,chr13,18538879,18538880,18551214,ENSG00000231238
4,ENSG00000231238,ENST00000448748,HGNC:42667,LINC00349,18538880,18551214,13,chr13,18538879,18538880,18551214,ENSG00000231238


In [26]:
df21 = df32[['0','1','2','3','4']]

In [27]:
df21 = df21.sort_values(by=['0','1'],ascending=True)

In [28]:
new_header = df21.iloc[0] 
df21 = df21[1:] 
df21.columns = new_header 


In [29]:
df21.head()

215699,chr1,11868,11869,14409,ENSG00000223972
215700,chr1,11868,11869,14409,ENSG00000223972
215710,chr1,14403,14404,29570,ENSG00000227232
215711,chr1,17368,17369,17436,ENSG00000278267
215712,chr1,29553,29554,31109,ENSG00000243485
215713,chr1,29553,29554,31109,ENSG00000243485


In [30]:
df21.to_csv('tempGene.bed', sep="\t",index=False)

In [32]:
# allEnv = checkIfgeneOrEnhancer(rawBed,'tempGene.bed')
allEnv = checkIfgeneOrEnhancer('ENCFF979PSS.bed.gz','tempGene.bed')

In [ ]:
df1_isEnhancer = allEnv[0]
df1_isGene = allEnv[1]
df2_isEnhancer = allEnv[2]
df2_isGene = allEnv[3]

In [ ]:
print("before deletion")

In [ ]:
del(allEnv)
del(df21)
del(df32)
del(df212)
del(df213)

In [ ]:
print("after deletion")

In [ ]:
df_all = pd.merge(df1_isEnhancer, df2_isGene , how='inner',  left_on=[14], right_on = [8])

In [ ]:
print("after deletion21")

In [ ]:
df_all1 = pd.merge(df2_isEnhancer, df1_isGene , how='inner',  left_on=[14], right_on = [8])

In [ ]:
print("after deletion2")

In [ ]:
len(df1_isEnhancer.columns)
len(df2_isGene.columns)

In [ ]:
df_final = pd.concat([df_all, df_all1], axis=0)

In [ ]:
del(df_all)
del(df_all1)

In [ ]:
df_final.head()

In [ ]:
k = df_final['9_y']

In [ ]:
df_final.head()

In [ ]:
df_final.drop(columns=df_final.columns[4:len(df1_isEnhancer.columns)], 
        axis=1, 
        inplace=True)
df_final.drop(columns=df_final.columns[9:len(df2_isGene.columns)+len(df1_isEnhancer.columns)], 
        axis=1, 
        inplace=True)
df_final.drop(columns=df_final.columns[5], 
        axis=1, 
        inplace=True)

In [ ]:
df_final.head()

In [ ]:
df_final[5] = k

In [ ]:
df_final = df_final.drop_duplicates(subset=['3_x', '4_y'], keep='last')

In [ ]:
df_final.head()

In [ ]:
new_header = df_final.iloc[0] 
df_final = df_final[1:] 
df_final.columns = new_header 

In [ ]:
df_final.head()

In [ ]:
df_final.to_csv('tempFinalchiaPet.bed',sep='\t',header=None,index=False)

In [ ]:
print("chiapet printed")

In [ ]:
df_final = pd.read_csv('tempFinalchiaPet.bed',sep='\t',header=None)

In [ ]:
df_final.head(50)

In [ ]:
df_final[df_final[3] =='EH38E2781077']

In [ ]:
df_final[df_final[7]=='EH38E2781080']

In [ ]:
df_final.head()

In [ ]:
df_final[df_final[3].isin(['EH38E2836060','EH38E2836065','EH38E1382733','EH38E2836036','EH38E2836028','EH38E2835994'])]